https://medium.com/google-cloud/gemma-3-ollama-on-colab-a-developers-quickstart-7bbf93ab8fef

# Install and settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab_Notebooks/1140522 Gemma3'

/content/drive/MyDrive/Colab_Notebooks/1140522 Gemma3


In [ ]:
!sudo apt update && sudo apt install pciutils lshw
!pip install ollama

In [ ]:
#@title Install Ollama
#使用 curl 下載安裝腳本 install.sh。
#透過 | 把下載到的腳本直接交由 sh 執行
!curl -fsSL https://ollama.com/install.sh | sh  #安裝到colab虛擬機


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#您可以透過以下指令，在 Google Colab 或任何 Linux 環境中啟動 Ollama 伺服器，
# 並確保它在背景執行(&)、不受目前命令列中斷影響，且將輸出記錄於 ollama.log：
!nohup ollama serve > ollama.log 2>&1 &

# Run Gemma3

In [ ]:
#首次執行特定的語言模型（如 gemma3:12b）時，Ollama 需要從網路上下載該模型的「權重參數」。
#由於模型體積通常相當龐大，這個過程可能需要一段時間，實際所需時間取決於：
!ollama run gemma3:12b “What is the capital of the Netherlands?”


The capital of the Netherlands is **Amsterdam**.



While Amsterdam is the capital and largest city, it's important to note that **The Hague (Den Haag)** is the seat of the government and home to the International Court of Justice. So, it has a different kind of significance!



In [ ]:
#解釋圖片
!ollama run gemma3:12b "Describe what's happening in this image: demo.png"

Okay, I've analyzed the image "demo.png". Here's a description of what's happening:

**Overall Scene:**

The image depicts a brightly lit, stylized illustration or infographic related to programming or software development. It appears to be a metaphorical representation of a development process.

**Key Elements & Their Actions:**

*   **A Robotic Arm:** A metallic, robotic arm is extending from the left side of the image. It's holding a small, glowing, blue "code fragment" (likely representing a piece of code).
*   **A Tower/Structure:** The robotic arm is delivering the code fragment into a tall, complex tower-like structure composed of various geometric shapes (cubes, cylinders, spheres) and connecting lines. This tower visually symbolizes a software project or application, with the different elements representing modules, functionalities, or architecture.
*   **Glowing Progress/Indicators:** Within the tower structure, there are glowing indicators/circles. They suggest that the code

## 以python程式碼方式執行

In [ ]:
import ollama

try:
    response = ollama.generate(
        model="gemma3:12b",
        prompt="What is Friesland?"
    )
    print(response["response"])

except Exception as e:
    print(f"An error occurred: {e}")
    # Optional: Check the ollama.log file for server-side issues

Friesland is a fascinating place with multiple layers of meaning. Here's a breakdown of what Friesland refers to, encompassing geography, culture, and history:

**1. Friesland (The Netherlands - "Friesland"):**

*   **Province of the Netherlands:** This is the most common understanding of Friesland. It's a province in the northern Netherlands, known for its distinct culture, language, and traditions.
    *   **Geography:** It has a unique landscape of canals, lakes, and farmland, largely below sea level.  It's known for its "elf hills" - hills formed by glacial deposits.
    *   **Capital:** Leeuwarden (also known as 's-Gravenzande)
    *   **Language:** Frisian (Frysk). This is a Germanic language closely related to English and the Frisian languages of Germany. It's officially recognized and spoken by a significant portion of the population.
    *   **Culture:**  Frisian culture is strong and fiercely protected.  They have unique music, dance, art, and traditional costumes. The "vrijh

In [42]:
#Streaming Responses
import ollama

try:
    client = ollama.Client()
    stream = client.generate(
        model="gemma3:12b",
        prompt="Explain the theory of relativity in one concise paragraph.",
        stream=True
    )

    print("Gemma 3 Streaming:")
    for chunk in stream:
        print(chunk['response'], end='', flush=True)
    print() # Newline after streaming finishes

except Exception as e:
    print(f"An error occurred during streaming: {e}")

Gemma 3 Streaming:
An error occurred during streaming: [Errno 111] Connection refused


# Building a Simple Chatbot

`client.chat()` 方法透過維護訊息歷史紀錄來支援多輪對話。以下範例建立了一個基本的命令列聊天介面，當使用者輸入exit才會中止對話


In [ ]:
import ollama

try:
    client = ollama.Client()
    messages = [] # Stores the conversation history

    print("Starting chat with Gemma 3 (type 'exit' to quit)")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Exiting chat.")
            break

        # Append user message to history
        messages.append({
            'role': 'user',
            'content': user_input
        })

        # Get streaming response from the model
        response_stream = client.chat(
            model='gemma3:12b',
            messages=messages,
            stream=True #即時對話顯示
        )

        print("Gemma: ", end="")
        full_assistant_response = ""
        # Process and print the streamed response
        for chunk in response_stream:
            token = chunk['message']['content']
            print(token, end='', flush=True)
            full_assistant_response += token
        print() # Newline after assistant finishes

        # Append assistant's full response to history
        messages.append({
            'role': 'assistant',
            'content': full_assistant_response
        })

except Exception as e:
    print(f"\nAn error occurred during chat: {e}")

Starting chat with Gemma 3 (type 'exit' to quit)
You: 台灣哪裡好玩
Gemma: 台灣好玩的地方真的太多了！以下我將依照不同主題分類，提供一些建議，你可以依照自己的喜好選擇：

**一、自然風光類：**

*   **太魯閣國家公園 (花蓮)：** 壯麗的峽谷地形，步道多樣，適合健行、欣賞大自然。
*   **阿里山國家風景區 (嘉義)：** 日出、雲海、森林、櫻花，多樣化的自然景觀。
*   **日月潭 (南投)：** 台灣最大的湖泊，湖光山色，可以搭船遊湖、騎自行車。
*   **墾丁國家公園 (屏東)：** 海灘、珊瑚礁、熱帶植物，適合水上活動、陽光浴。
*   **玉山國家公園 (南投、嘉義)：** 台灣最高峰，適合登山健行，欣賞高山景觀。
*   **清水斷崖 (台南)：** 壯觀的懸崖峭壁，海蝕地形奇特，欣賞落日美景。
*   **合歡山 (南投)：** 高山景觀，觀星、賞雪的絕佳地點。
*   **澎湖群島 (澎湖)：** 離島風光、美麗沙灘、豐富海洋生態。
*   **綠島 (台東)：** 火山島嶼，潛水、浮潛、探索地質景觀。
*   **小琉球 (屏東)：** 珊瑚礁島，適合浮潛、潛水、體驗海洋生態。

**二、文化古蹟類：**

*   **台北故宮博物院 (台北)：** 收藏豐富的中國文物藝術品。
*   **九份 (新北市)：** 充滿懷舊氣息的山城，狹窄的石板路、古老的建築，感受金礦時期的風貌。
*   **十分瀑布 (新北市)：** 壯觀的瀑布景觀，可以放天燈祈福。
*   **淡水老街 (新北市)：** 充滿歷史感的街道，品嚐小吃、欣賞夕陽。
*   **鹿港老街 (彰化)：** 保存完整的傳統街道，體驗傳統文化、品嚐小吃。
*   **安平古堡 (台南)：** 台灣歷史最悠久的城堡之一，感受歷史滄桑。
*   **赤崁樓 (台南)：** 充滿歷史意義的古蹟，感受文化底蘊。
*   **保安宮 (台南)：** 精美的建築雕刻，感受傳統宗教文化。

**三、城市觀光類：**

*   **台北 101 (台北)：** 曾經的世界最高樓，俯瞰台北市景。
*   **西門町 (台北)：** 年輕人的聚集地，流行文化、美食、購物。
*   **信義區 (台北)：** 現代化的商業區，高級購物